<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import gensim
# import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

# from collections import Counter

import warnings
warnings.filterwarnings('ignore')

C:\Users\John\Anaconda3\envs\U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [3]:
df = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [4]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [5]:
features = ['name', 'brand', 'categories', 'primaryCategories', 'reviews.rating', 'reviews.text']
df = df[features]

In [6]:
df.head()

,name,brand,categories,primaryCategories,reviews.rating,reviews.text
0,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,3,I order 3 of them and one of the item is bad q...
1,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,4,Bulk is always the less expensive way to go fo...
2,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,5,Well they are not Duracell but for the price i...
3,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,5,Seem to work as well as name brand batteries a...
4,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,5,These batteries are very long lasting the pric...


In [7]:
df.shape

(28332, 6)

In [8]:
tokens = []

for doc in tokenizer.pipe(df['reviews.text'], batch_size=500):

    doc_tokens = []

    for token in doc: 
        if token.text.lower() not in nlp.Defaults.stop_words:
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [9]:
id2word = corpora.Dictionary(df['tokens'])

In [10]:
id2word.token2id['batteries']

20

In [11]:
id2word[869]

'turned'

In [12]:
len(tokens)

28332

In [13]:
len(id2word)

21224

In [14]:
id2word

In [15]:
import sys
sys.getsizeof(id2word)

56

In [16]:
id2word.filter_extremes(no_below=5, no_above=1)

In [17]:
len(id2word)

5564

In [18]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [19]:
corpus[869]

[(18, 1),
 (20, 1),
 (24, 1),
 (27, 1),
 (153, 1),
 (157, 1),
 (167, 1),
 (470, 1),
 (619, 1),
 (753, 1),
 (1284, 1),
 (1434, 1),
 (1777, 1),
 (2066, 1)]

In [20]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=8
                  )

In [21]:
lda.print_topics()

[(0,
  '0.032*"great" + 0.032*"easy" + 0.023*"use" + 0.016*"tablet" + 0.015*"it\'s" + 0.012*"excellent" + 0.010*"light" + 0.010*"perfect" + 0.009*"tablet." + 0.008*"best"'),
 (1,
  '0.041*"great" + 0.025*"batteries" + 0.022*"price" + 0.016*"battery" + 0.015*"work" + 0.014*"i\'ve" + 0.013*"better" + 0.013*"best" + 0.011*"buy" + 0.011*"life"'),
 (2,
  '0.021*"amazon" + 0.020*"google" + 0.019*"tablet" + 0.017*"best" + 0.016*"store" + 0.013*"play" + 0.012*"tablet." + 0.011*"apps" + 0.011*"good" + 0.009*"easy"'),
 (3,
  '0.034*"fire" + 0.032*"tablet" + 0.024*"bought" + 0.019*"works" + 0.018*"great" + 0.016*"amazon" + 0.014*"hd" + 0.012*"price" + 0.009*"screen" + 0.009*"love"'),
 (4,
  '0.014*"battery" + 0.013*"love" + 0.012*"purchased" + 0.011*"batteries" + 0.011*"use" + 0.010*"happy" + 0.010*"good" + 0.010*"didn\'t" + 0.010*"price" + 0.009*"i\'m"'),
 (5,
  '0.126*"great" + 0.104*"good" + 0.039*"price." + 0.035*"product" + 0.031*"price" + 0.019*"value" + 0.014*"tablet" + 0.013*"price!" + 0.

In [22]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [23]:
topics = [' '.join(t[0:5]) for t in words]

In [24]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
great easy use tablet it's

------ Topic 1 ------
great batteries price battery work

------ Topic 2 ------
amazon google tablet best store

------ Topic 3 ------
fire tablet bought works great

------ Topic 4 ------
battery love purchased batteries use

------ Topic 5 ------
great good price. product price

------ Topic 6 ------
great tablet works it's bought

------ Topic 7 ------
tablet kids year time love

------ Topic 8 ------
batteries long batteries. like work

------ Topic 9 ------
loves love it. bought old

------ Topic 10 ------
great work good batteries. deal

------ Topic 11 ------
kindle it's great tablet price

------ Topic 12 ------
easy tablet use love kindle

------ Topic 13 ------
kindle battery great it's amazon

------ Topic 14 ------
batteries amazon don't long brand



In [25]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [26]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.246999 -0.050948       1        1  11.359076
9     -0.175150  0.022203       2        1   9.970578
14     0.188214 -0.080219       3        1   8.673433
12    -0.174924 -0.029357       4        1   7.813109
7     -0.127619  0.000169       5        1   7.329999
11    -0.090025 -0.026484       6        1   7.235950
13    -0.031672 -0.108473       7        1   6.844007
3     -0.080553 -0.026619       8        1   6.729357
5      0.067538  0.219775       9        1   6.289667
2     -0.047532 -0.041387      10        1   5.625543
10     0.190937  0.066637      11        1   5.115844
0     -0.077729  0.041958      12        1   4.879297
6     -0.068699  0.083379      13        1   4.361249
1      0.129805  0.019018      14        1   3.966698
4      0.050411 -0.089652      15        1   3.806194, topic_info=     Category         Freq        Term        Total  loglift  logprob
18    Default  5286.000000   batteries  5286.000000  30.0000  30.0000
75    Default  7682.000000       great  7682.000000  29.0000  29.0000
64    Default  4774.000000        good  4774.000000  28.0000  28.0000
2633  Default  2200.000000       loves  2200.000000  27.0000  27.0000
3536  Default  4884.000000      tablet  4884.000000  26.0000  26.0000
129   Default  1408.000000  batteries.  1408.000000  25.0000  25.0000
995   Default  2804.000000        easy  2804.000000  24.0000  24.0000
21    Default  1838.000000        work  1838.000000  23.0000  23.0000
60    Default  1366.000000      price.  1366.000000  22.0000  22.0000
114   Default  1631.000000        year  1631.000000  21.0000  21.0000
212   Default  1941.000000        kids  1941.000000  20.0000  20.0000
315   Default  1272.000000     product  1272.000000  19.0000  19.0000
17    Default  2608.000000       price  2608.000000  18.0000  18.0000
1050  Default  1906.000000        fire  1906.000000  17.0000  17.0000
1055  Default  1788.000000         old  1788.000000  16.0000  16.0000
2986  Default  2344.000000      kindle  2344.000000  15.0000  15.0000
568   Default   521.000000        deal   521.000000  14.0000  14.0000
502   Default  2489.000000         it.  2489.000000  13.0000  13.0000
20    Default   970.000000       brand   970.000000  12.0000  12.0000
24    Default  2102.000000        long  2102.000000  11.0000  11.0000
104   Default  1357.000000       works  1357.000000  10.0000  10.0000
98    Default  3371.000000        love  3371.000000   9.0000   9.0000
3     Default  1733.000000     battery  1733.000000   8.0000   8.0000
4206  Default   478.000000      google   478.000000   7.0000   7.0000
359   Default  1297.000000        best  1297.000000   6.0000   6.0000
478   Default   577.000000       value   577.000000   5.0000   5.0000
30    Default  3118.000000      bought  3118.000000   4.0000   4.0000
407   Default  2170.000000        it's  2170.000000   3.0000   3.0000
26    Default  3319.000000      amazon  3319.000000   2.0000   2.0000
1041  Default   967.000000        read   967.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
3     Topic15   179.652863     battery  1733.973511   1.0014  -4.2385
1398  Topic15    17.675777      played    37.143883   2.5259  -6.5574
501   Topic15   112.917885         i'm   959.784851   1.1285  -4.7029
1367  Topic15    30.270308     charged    96.566284   2.1085  -6.0194
293   Topic15   107.844528         new   902.040833   1.1446  -4.7489
218   Topic15    39.733070       long.   170.472748   1.8121  -5.7474
618   Topic15    42.026943      couple   194.690979   1.7354  -5.6913
109   Topic15   103.587677         got  1241.086914   0.7852  -4.7892
260   Topic15    53.417233     brands.   342.043488   1.4117  -5.4514
94    Topic15    64.622360       them.   514.505920   1.1939  -5.2610
98    Topic15   164.548706        love  3371.692871   0.2486  -4.3264
106   Topic15    41.077255   

In [27]:
lda[corpus[0]]

[(10, 0.43320844), (14, 0.4880036)]

In [30]:
distro = [lda[d] for d in corpus]

In [31]:
distro[0]

[(10, 0.43340206), (14, 0.48780993)]

In [32]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [33]:
len(new_distro)

28332

In [41]:
df = pd.DataFrame.from_records(new_distro)
df.columns = topics

# because I don't know how any of this works, I can't get rating into the new configuration

# df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [42]:
df.head()

,great easy use tablet it's,great batteries price battery work,amazon google tablet best store,fire tablet bought works great,battery love purchased batteries use,great good price. product price,great tablet works it's bought,tablet kids year time love,batteries long batteries. like work,loves love it. bought old,great work good batteries. deal,kindle it's great tablet price,easy tablet use love kindle,kindle battery great it's amazon,batteries amazon don't long brand
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.433226,0.000000,0.000000,0.000000,0.487986
1,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844444,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111
2,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766666,0.016667,0.016667,0.016667,0.016667
3,0.011111,0.844444,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111
4,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844444,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111


## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling